# Grundlegende Infos

**Aufgabe:**
Zusammenführen der Ergebnis-Dateien

**Eingabedateien:** XLSX-Dateien

# Notwendige Imports & Vorbereitung
## Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import os
import sqlite3

## Anlegen der SQLite Datenbank

In [2]:
conn = sqlite3.connect("../working/db.sqlite")

stmt = """
create table athletes
(
    id         integer
        constraint athletes_pk
            primary key autoincrement,
    name       TEXT,
    first_name TEXT,
    gender     TEXT,
    year       TEXT,
    club       TEXT,
    category   TEXT
);
"""

conn.execute(stmt)

# Einstellungen
## Verzeichnisse

In [3]:
input_dir = "../result"
output_dir = "../finished"

# Notwendige Funktionen

**ID des Athleten**

In [4]:
def get_athlete_id (name, first_name, year):
    stmt = """
        SELECT id from athletes WHERE name=? AND first_name=? AND year=?
    """
    c = conn.cursor()
    result = c.execute(stmt, (name, first_name, year))
    id = c.fetchone()
    c.close()
    return id

**Einfügen eines Athleten**

In [13]:
def insert_athlete (name, first_name, year, gender, club, category):
    # Check if athlete exists
    if get_athlete_id(name, first_name, year) is not None:
        return

    # Insert Athlete
    c = conn.cursor()

    stmt = """
    insert into athletes (name, first_name, gender, "year", club, category) values (?, ?, ?, ?, ?, ?);
    """

    c.execute(stmt, (name, first_name, gender, year, club, category))
    conn.commit()
    c.close()

# Einlesen der Athleten

In [16]:
row = []

for file in os.listdir(input_dir):
    # Ignore open files
    if file.startswith("~"):
        continue

    file_path = os.path.join(input_dir, file)

    for gender in ("male", "female"):
        df_excel = pd.read_excel(file_path, gender, index_col=0)

        for index, row in df_excel.iterrows():
            insert_athlete(row["Nachname"], row["Vorname"], row["Geb. Jahr"], gender, row["Verein"], row["Altersklasse"])

# Einlesen der Ergebnisse